In [34]:
import numpy as np
import math
import time

## 1.1 Ejercicios Básicos

### 1 Procesamiento de visibilidades

In [35]:
# 1.1 Ejercicios Básicos
## 1 Procesamiento de visibilidades
def visilities(n=1_000_000, dtype=np.int32):
    amplitud = np.random.uniform(0, 1, n).astype(dtype)
    fase = np.random.uniform(0, 2 * np.pi, n).astype(dtype)
    visibilidades = amplitud * np.exp(1j * fase)
    return visibilidades


def prom_power(v):
    prom = []
    for value in v:
        real = value.real
        imag = value.imag
        p = math.sqrt(pow(real,2) + pow(imag,2))
        prom.append(p)

    sum = 0 
    for n in prom:
        sum += n
    prom = sum / len(v)
    return prom

def prom_power_vectorized(v):
    return np.mean(np.abs(v))

In [36]:
%%time
v = visilities()

CPU times: total: 46.9 ms
Wall time: 42.8 ms


In [37]:
%%time
prom = prom_power(v)

CPU times: total: 312 ms
Wall time: 273 ms


In [38]:
%%time
prom_np = prom_power_vectorized(v)

CPU times: total: 31.2 ms
Wall time: 3.02 ms


### 2 Optimizacion de memoria en Datos astronómicos

In [39]:
## 2 Optimizacion de memoria en Datos astronómicos
v_int32 = visilities(1000)
v_int64 = visilities(1000, dtype=np.int64)
v_float32 = visilities(1000, dtype=np.float32)
v_float64 = visilities(1000,dtype=np.float64)

print(f"Memoria I32: {v_int32.nbytes / 1e6} MB")
print(f"Memoria I64: {v_int64.nbytes / 1e6} MB")
print(f"Memoria F32: {v_float32.nbytes / 1e6} MB")
print(f"Memoria F64: {v_float64.nbytes / 1e6} MB")

Memoria I32: 0.016 MB
Memoria I64: 0.016 MB
Memoria F32: 0.008 MB
Memoria F64: 0.016 MB


In [40]:
%%time
test1 = v_int32 * np.random.randint(1, 10, dtype=np.int32)

CPU times: total: 0 ns
Wall time: 71.5 μs


In [41]:
%%time
test2 = v_int64 * np.random.randint(1, 10, dtype=np.int64)

CPU times: total: 0 ns
Wall time: 57.5 μs


In [42]:
%%time
test3 = v_float32 * np.float32(np.random.randint(1, 10))

CPU times: total: 0 ns
Wall time: 78.9 μs


In [43]:
%%time
test4 = v_float64 * np.float64(np.random.randint(1, 10))

CPU times: total: 0 ns
Wall time: 80.6 μs


### 3 Vistas vs Copias

In [49]:
array_original = np.array([1,2,3,4,5,6,7,8,9,10])

vista = array_original[1:5]
copia = array_original[1:5].copy()

vista[2] = 88
copia[2] = 89

print(f"Original {array_original}")
print(f"Vista {vista}")
print(f"Copia {copia}")

Original [ 1  2  3 88  5  6  7  8  9 10]
Vista [ 2  3 88  5]
Copia [ 2  3 89  5]


Al utilizar una vista, no se está creando un arreglo nuevo, solamante se está haciendo una ventana que apunta a directamente a la memoria del arreglo original. Por eso se modifican los valores del arreglo original. copy() si crea un arreglo completamente nuevo.

### 4 Análisis de Rendimiento


In [ ]:
def norma_loop(x):
    suma = 0.0
    for i in x:
        suma += i * i
    return suma ** 0.5

def norma_numpy_vec(x):
    return np.sqrt(np.sum(x ** 2))

def norma_numpy_func(x):
    return np.linalg.norm(x)

def medir_tiempo(func, x, n_reps=5):
    tiempos = []
    for _ in range(n_reps):
        inicio = time.perf_counter()
        func(x)
        fin = time.perf_counter()
        tiempos.append(fin - inicio)
    return np.mean(tiempos)

sizes = [10**3, 10**5, 10**7]

resultados = []

for n in sizes:
    x = np.random.rand(n)
    t_loop = medir_tiempo(norma_loop, x)
    t_vec = medir_tiempo(norma_numpy_vec, x)
    t_func = medir_tiempo(norma_numpy_func, x)
    resultados.append((n, t_loop, t_vec, t_func))

# Mostrar tabla de resultados
print(f"{'Tamaño':>10} | {'Loop Python':>12} | {'Numpy Vec':>10} | {'np.linalg.norm':>15}")
print("-" * 55)
for n, t1, t2, t3 in resultados:
    print(f"{n:10d} | {t1:12.6f} | {t2:10.6f} | {t3:15.6f}")

    Tamaño |  Loop Python |  Numpy Vec |  np.linalg.norm
-------------------------------------------------------
      1000 |     0.000086 |   0.000036 |        0.000010
    100000 |     0.007246 |   0.000096 |        0.000089
  10000000 |     0.726855 |   0.018562 |        0.001663


### 5 Manipulacion de Arrays

In [51]:
matriz = np.random.rand(1000, 1000)

def maximo_por_fila(matriz):
  return np.max(matriz, axis=1)

maximos_filas = maximo_por_fila(matriz)

print("Máximo de las primeras 5 filas:", maximos_filas[:5])

def maximo_por_columna(matriz):
  return np.max(matriz, axis=0)

maximos_columnas = maximo_por_columna(matriz)

print("Máximo de las primeras 5 columnas:", maximos_columnas[:5])

def normalizar_filas(matriz):
                     
  media_filas = np.mean(matriz, axis=1, keepdims=True)
  std_filas = np.std(matriz, axis=1, keepdims=True)
  std_filas[std_filas == 0] = 1  
  matriz_normalizada = (matriz - media_filas) / std_filas

  return matriz_normalizada

matriz_norm = normalizar_filas(matriz)

print("Media de la primera fila normalizada:", np.mean(matriz_norm[0, :]))
print("Desviación estándar de la primera fila normalizada:", np.std(matriz_norm[1, :]))

# PYTHON

def max_fila_python(matriz):
    maximos = []
    for fila in matriz:
        maximos.append(max(fila))
    return maximos

def normalizar_filas_python(matriz):
    matriz_norm = []
    for fila in matriz:
        media = sum(fila) / len(fila)
        varianza = sum([(x - media) ** 2 for x in fila]) / len(fila)
        std = varianza ** 0.5
        if std == 0:
            std = 1
        fila_norm = [(x - media) / std for x in fila]
        matriz_norm.append(fila_norm)
    return matriz_norm

matriz = np.random.rand(1000, 1000)
matriz_lista = matriz.tolist()


print("\n")


inicio = time.time()
maximo_por_fila(matriz)
fin = time.time()
print(f"NumPy - Máximo por fila: {fin - inicio:.6f} segundos")

# Versión Python
inicio = time.time()
max_fila_python(matriz_lista)
fin = time.time()
print(f"Python puro - Máximo por fila: {fin - inicio:.6f} segundos")

print("-" * 30)
inicio = time.time()
normalizar_filas(matriz)
fin = time.time()
print(f"NumPy - Normalizar filas: {fin - inicio:.6f} segundos")

# Versión Python
inicio = time.time()
normalizar_filas_python(matriz_lista)
fin = time.time()
print(f"Python puro - Normalizar filas: {fin - inicio:.6f} segundos")

Máximo de las primeras 5 filas: [0.99953202 0.99977242 0.99982024 0.99926854 0.99938548]
Máximo de las primeras 5 columnas: [0.99959313 0.9990345  0.99807854 0.99922234 0.99944385]
Media de la primera fila normalizada: 9.85878045867139e-17
Desviación estándar de la primera fila normalizada: 0.9999999999999999


NumPy - Máximo por fila: 0.000527 segundos
Python puro - Máximo por fila: 0.007099 segundos
------------------------------
NumPy - Normalizar filas: 0.007976 segundos
Python puro - Normalizar filas: 0.092153 segundos


## Broadcasting

### 6 broadcasting basico


### 7 Broadcasting con Arrays 2D

### 8 None y np.newaxis

In [73]:
array1D = np.array(np.random.randint(1,10, size=6))

print(f"array: {array1D}\n")

matrix6x1 = array1D[:,None]
matrix1x6 = array1D[None, :]

print(f"6x1: {matrix6x1} \n shape: {matrix6x1.shape}")

print(f"1x6: {matrix1x6}\n shape: {matrix1x6.shape}\n")

print("suma:\n", matrix6x1 + matrix1x6)

print("mult:\n", matrix6x1 * matrix1x6)


array: [9 8 7 3 2 3]

6x1: [[9]
 [8]
 [7]
 [3]
 [2]
 [3]] 
 shape: (6, 1)
1x6: [[9 8 7 3 2 3]]
 shape: (1, 6)

suma:
 [[18 17 16 12 11 12]
 [17 16 15 11 10 11]
 [16 15 14 10  9 10]
 [12 11 10  6  5  6]
 [11 10  9  5  4  5]
 [12 11 10  6  5  6]]
mult:
 [[81 72 63 27 18 27]
 [72 64 56 24 16 24]
 [63 56 49 21 14 21]
 [27 24 21  9  6  9]
 [18 16 14  6  4  6]
 [27 24 21  9  6  9]]


Como los shapes de los arreglos no son iguales, numpy tiene que revisar las dimensiones de derecha a izquierda en este caso tenemos (6,1) y (1,6). Como el 1 y 6 no coinciden numpy estira el arreglo a la dimension mas grande, por tanto el resultado siempre es de 6x6. Para el caso del vector columna estira hacia la derecha agregando mas columnas con los mismos valores de cada fila, en el caso del vector fila es lo mismo.

### 9 Cálculo de Baselines del Interferómetro